In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

import lightning as L
from torch.utils.data import DataLoader, TensorDataset


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\Alosiuis Akonteh\Documents\Capstone\FPGA_LSTM_Accel\alloy_env\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\Alosiuis Akonteh\Documents\Capstone\FPGA_LSTM_Accel\alloy_env\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\Alosiuis Akonteh\Documents\Capstone\FPGA_LSTM_Accel\alloy

In [65]:

num_elements  = 4
class LSTMbyHand(L.LightningModule):
    def __init__(self):
        super().__init__()
        
        # use normal distribution to generate random weights and biases
        mean = torch.tensor(0.0)
        std = torch.tensor(1.0)

        # short term memory weights and biases
        self.wlr1 = nn.Parameter(torch.normal(mean=mean, std=std, size=(num_elements, num_elements)), requires_grad=True)
        self.wlr2 = nn.Parameter(torch.normal(mean=mean, std=std, size=(num_elements, num_elements)), requires_grad=True)
        self.blr1 = nn.Parameter(torch.zeros(num_elements), requires_grad=True)

        # candidate long term memory/cell state weights and biases
        self.wpr1 = nn.Parameter(torch.normal(mean=mean, std=std, size=(num_elements,num_elements)), requires_grad=True)
        self.wpr2 = nn.Parameter(torch.normal(mean=mean, std=std, size=(num_elements,num_elements)), requires_grad=True)
        self.bpr1 = nn.Parameter(torch.zeros(num_elements), requires_grad=True)

        # potential memory weights and biases
        self.wp1 = nn.Parameter(torch.normal(mean=mean, std=std, size=(num_elements, num_elements)), requires_grad=True)
        self.wp2 = nn.Parameter(torch.normal(mean=mean, std=std, size=(num_elements, num_elements)), requires_grad=True)
        self.bp1 = nn.Parameter(torch.zeros(num_elements), requires_grad=True)

        # output weights and biases
        self.wo1 = nn.Parameter(torch.normal(mean=mean, std=std, size=(num_elements,num_elements)), requires_grad=True)
        self.wo2 = nn.Parameter(torch.normal(mean=mean, std=std, size=(num_elements,num_elements)), requires_grad=True)
        self.bo1 = nn.Parameter(torch.zeros(num_elements), requires_grad=True)

    def input_format(self, wsh, win, b):
        wsh = wsh.tolist()
        win = win.tolist()
        b = b.tolist()
        for i, j in enumerate(wsh):
            for l, k in enumerate(j):
                wsh[i][l] = int(wsh[i][l]*1000)

        for i, j in enumerate(win):
            for l, k in enumerate(j):
                win[i][l] = int(win[i][l]*1000)

        for i, j in enumerate(b):
            b[i] = int(b[i]*1000)

        data = {
            "input_weights": win,
            "gate_biases": b,
            "short_weights": wsh
        }
        return data
    
    def lstm_unit(self, input_value, long_memory, short_memory):
        # determine how much of long term memory to remember - forget gate
        long_remember_percent = torch.sigmoid(torch.matmul(short_memory.unsqueeze(0), self.wlr1)
                                              + torch.matmul(input_value.unsqueeze(0),  self.wlr2) +
                                              self.blr1).reshape(-1)
        # print(f"forget = {(short_memory * self.wlr1)+ (input_value * self.wlr2) +self.blr1}")
        # calculate how much of potential memory should be remembered - Candidate gate
        potential_remember_percent = torch.sigmoid(torch.matmul(short_memory.unsqueeze(0),  self.wpr1)
                                              + torch.matmul(input_value.unsqueeze(0),  self.wpr2) +
                                              self.bpr1).reshape(-1)
        # print(f"candidate = {(short_memory * self.wpr1)  + (input_value * self.wpr2) + self.bpr1}")
        # calculate candidate memory - Input gate
        potential_memory = torch.tanh(torch.matmul(short_memory.unsqueeze(0), self.wp1)
                                              + torch.matmul(input_value.unsqueeze(0), self.wp2) +
                                              self.bp1).reshape(-1)
        # print(f"input = {(short_memory * self.wp1)+ (input_value * self.wp2) +self.bp1}")
        # update long term memory
        updated_long_memory = ((long_memory * long_remember_percent) +
                               (potential_remember_percent * potential_memory)).reshape(-1)
        print(f"new_long = {updated_long_memory}")
        # calculate new short term memory and how much of it to output - Output gate
        output_percent = torch.sigmoid(torch.matmul(short_memory.unsqueeze(0), self.wo1) + 
                                       torch.matmul(input_value.unsqueeze(0), self.wo2) +
                                       self.bo1).reshape(-1)
        # print(f"output = {(short_memory * self.wo1) + (input_value * self.wo2) +self.bo1}")
        updated_short_memory = (torch.tanh(updated_long_memory) * output_percent).reshape(-1)
        print(f"new short = {updated_short_memory}")
        print(f"\n")

        return ([updated_long_memory, updated_short_memory])
    
    def forward(self, input):
        long_memory = torch.zeros(num_elements).unsqueeze(0)
        short_memory = torch.zeros(num_elements).unsqueeze(0)
        # test inputs
        day1 = input[0]
        day2 = input[1]
        day3 = input[2]
        day4 = input[3]

        long_memory, short_memory = self.lstm_unit(day1, long_memory, short_memory)
        long_memory, short_memory = self.lstm_unit(day2, long_memory, short_memory)
        long_memory, short_memory = self.lstm_unit(day3, long_memory, short_memory)
        long_memory, short_memory = self.lstm_unit(day4, long_memory, short_memory)

        return short_memory

    def configure_optimizers(self):
        return Adam(self.parameters())

    def training_step(self, batch, batch_idx):
        input_i, label_i = batch
        output_i = self.forward(input_i[0])
        loss = (output_i - label_i)**2

        self.log("train_loss ", loss)
        
        if (label_i == 0):
            self.log("Out 0", output_i)
        else:
            self.log("Out 1", output_i)
        return loss


In [40]:
a = torch.tensor([0., 0.25, 0.5, 1.]).unsqueeze(0)
print(a.size())

torch.Size([1, 4])


In [66]:

model = LSTMbyHand()
print("Company A: observed = 0, Predicted = ", model(torch.tensor([[0., 0.25, 0.5, 1.], [0., 0.25, 0.5, 1.], [0., 0.25, 0.5, 1.], [0., 0.25, 0.5, 1.]])).detach())

new_long = tensor([ 0.7195, -0.2836,  0.4899,  0.6760], grad_fn=<ViewBackward0>)
new short = tensor([ 0.5784, -0.2128,  0.2728,  0.3392], grad_fn=<ViewBackward0>)


new_long = tensor([ 1.2185, -0.7736,  0.8424,  0.9557], grad_fn=<ViewBackward0>)
new short = tensor([ 0.7834, -0.5703,  0.2910,  0.3944], grad_fn=<ViewBackward0>)


new_long = tensor([ 1.1725, -1.1970,  1.1668,  1.4254], grad_fn=<ViewBackward0>)
new short = tensor([ 0.7688, -0.7391,  0.3038,  0.3243], grad_fn=<ViewBackward0>)


new_long = tensor([ 1.0017, -1.4780,  1.4538,  2.0288], grad_fn=<ViewBackward0>)
new short = tensor([ 0.7120, -0.7906,  0.3405,  0.2878], grad_fn=<ViewBackward0>)


Company A: observed = 0, Predicted =  tensor([ 0.7120, -0.7906,  0.3405,  0.2878])


In [57]:
model = LSTMbyHand()
print("Company B: observed = 0.5, Predicted = ", model(torch.tensor([[1., 0.25, 0.5, 1.], [0., 0.25, 0.5, 1.], [0., 0.25, 0.5, 1.], [0., 0.25, 0.5, 1.]])).detach())

new_long = tensor([ 0.0024, -0.1840,  0.5791, -0.1599], grad_fn=<ViewBackward0>)
new short = tensor([ 9.9631e-05, -2.3340e-02,  2.8997e-02, -1.4548e-01],
       grad_fn=<ViewBackward0>)


new_long = tensor([-0.0493, -0.1221,  0.9957,  0.2426], grad_fn=<ViewBackward0>)
new short = tensor([-0.0033, -0.0122,  0.0520,  0.2201], grad_fn=<ViewBackward0>)


new_long = tensor([-0.0774, -0.0193,  1.3846,  0.4347], grad_fn=<ViewBackward0>)
new short = tensor([-0.0031, -0.0038,  0.0671,  0.3668], grad_fn=<ViewBackward0>)


new_long = tensor([-0.0899,  0.0750,  1.7308,  0.5142], grad_fn=<ViewBackward0>)
new short = tensor([-0.0029,  0.0187,  0.0746,  0.4175], grad_fn=<ViewBackward0>)


Company B: observed = 0.5, Predicted =  tensor([-0.0029,  0.0187,  0.0746,  0.4175])


In [43]:
inputs = torch.tensor([[[0., 0.25, 0.5, 1.], [0., 0.25, 0.5, 1.], [0., 0.25, 0.5, 1.], [0., 0.25, 0.5, 1.]], [[1., 0.5, 0.25, 1.], [1., 0.5, 0.25, 1.], [1., 0.5, 0.25, 1.], [1., 0.5, 0.25, 1.]]])
labels = torch.tensor([[1., 0.5, 0.25, 1.], [1., 0.5, 0.25, 1.]])

In [44]:
dataset = TensorDataset(inputs, labels)
dataloader = DataLoader(dataset)

In [45]:
trainer = L.Trainer(max_epochs=3000)
trainer.fit(model, train_dataloaders=dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name         | Type | Params | Mode
---------------------------------------------
  | other params | n/a  | 144    | n/a 
---------------------------------------------
144       Trainable params
0         Non-trainable params
144       Total params
0.001     Total estimated model params size (MB)
c:\Users\Alosiuis Akonteh\Documents\Capstone\FPGA_LSTM_Accel\alloy_env\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
c:\Users\Alosiuis Akonteh\Documents\Capstone\FPGA_LSTM_Accel\alloy_env\Lib\site-packages\lightning\pytorch\loops\fit_loop.py:298: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Se

Epoch 0:   0%|          | 0/2 [00:00<?, ?it/s] new_long = tensor([-0.2921, -0.0916,  0.8963, -0.1187], grad_fn=<ViewBackward0>)
new short = tensor([-0.1439, -0.0434,  0.4655, -0.0779], grad_fn=<ViewBackward0>)


new_long = tensor([-0.4038, -0.0455,  1.6138, -0.0884], grad_fn=<ViewBackward0>)
new short = tensor([-0.2030, -0.0197,  0.6612, -0.0666], grad_fn=<ViewBackward0>)


new_long = tensor([-0.4054,  0.0295,  2.1404, -0.0362], grad_fn=<ViewBackward0>)
new short = tensor([-0.2001,  0.0128,  0.7402, -0.0287], grad_fn=<ViewBackward0>)


new_long = tensor([-0.3714,  0.0910,  2.5395,  0.0115], grad_fn=<ViewBackward0>)
new short = tensor([-0.1777,  0.0409,  0.7760,  0.0093], grad_fn=<ViewBackward0>)




ValueError: `self.log(train_loss , tensor([[1.3870, 0.2107, 0.2767, 0.9815]]))` was called, but the tensor must have a single element. You can try doing `self.log(train_loss , tensor([[1.3870, 0.2107, 0.2767, 0.9815]]).mean())`

In [46]:
print("Company B: observed = 0.5, Predicted = ", model.forward(torch.tensor([[1., 0.5, 0.25, 1.],[0., 0.25, 0.5, 1.],[0., 0.25, 0.5, 1.],[0., 0.25, 0.5, 1.]])).detach())

new_long = tensor([-0.0961, -0.0369,  0.9837,  0.0344], grad_fn=<ViewBackward0>)
new short = tensor([-0.0286, -0.0115,  0.6801,  0.0224], grad_fn=<ViewBackward0>)


new_long = tensor([-0.2061,  0.0700,  1.6349,  0.0496], grad_fn=<ViewBackward0>)
new short = tensor([-0.0970,  0.0347,  0.7002,  0.0404], grad_fn=<ViewBackward0>)


new_long = tensor([-0.2670,  0.1282,  2.1633,  0.0638], grad_fn=<ViewBackward0>)
new short = tensor([-0.1226,  0.0629,  0.7636,  0.0521], grad_fn=<ViewBackward0>)


new_long = tensor([-0.2798,  0.1654,  2.5711,  0.0859], grad_fn=<ViewBackward0>)
new short = tensor([-0.1253,  0.0813,  0.7948,  0.0709], grad_fn=<ViewBackward0>)


Company B: observed = 0.5, Predicted =  tensor([-0.1253,  0.0813,  0.7948,  0.0709])


In [69]:
print(model.input_format(model.wlr1, model.wlr2, model.blr1))
print(model.input_format(model.wpr1, model.wpr2, model.bpr1))
print(model.input_format(model.wp1, model.wp2, model.bp1))
print(model.input_format(model.wo1, model.wo2, model.bo1))

{'input_weights': [[-1654, -486, -142, 1539], [1535, -2049, 253, -346], [1097, 892, -11, -9], [1181, 238, 1550, 935]], 'gate_biases': [0, 0, 0, 0], 'short_weights': [[-2247, -230, -1326, -66], [1615, -423, -1022, -1351], [-33, 46, -600, 2063], [495, 1572, -528, 227]]}
{'input_weights': [[-545, 273, 1106, -1404], [1063, -1128, -1974, -207], [-311, -482, 1543, 627], [1075, 998, -202, 1171]], 'gate_biases': [0, 0, 0, 0], 'short_weights': [[-868, 1021, 77, -346], [-203, -109, -1647, -986], [1098, -1345, -318, -744], [1053, -313, -669, 988]]}
{'input_weights': [[-766, 1392, -676, 116], [-433, 346, 1506, -2652], [181, 387, 544, 1434], [1748, -777, 1126, 1157]], 'gate_biases': [0, 0, 0, 0], 'short_weights': [[247, -582, -713, -1257], [1454, 81, 1163, -1622], [69, -234, 1634, -12], [383, -1670, 872, -969]]}
{'input_weights': [[109, 678, -347, -836], [35, 818, 160, 56], [2472, 299, 151, -1184], [1474, 857, 292, 883]], 'gate_biases': [0, 0, 0, 0], 'short_weights': [[434, 1395, -971, -234], [181,

In [ ]:
x = list([[5, 9], [5, 6]])
s = type(x)
if (s == list):
    if type(x[0]) == list:
        print("list")
    else:
        print("int")
else:
    print((s))

list


In [ ]:
def values(input):
        output_string = ''
        if (type(input) == int):
            output_string += str(input)
        elif type(input) == list:
            output_string = '('
            if type(input[0]) == int:
                for k, i in enumerate(input):
                    print(i)
                    output_string += str(i)
                    if k == len(input)-1:
                        break
                    output_string += ", "
                output_string += ')'

            elif type(input[0]) == list:
                for k, i in enumerate(input):
                    output_string += '('
                    for inner, j in enumerate(i):
                        output_string += str(j)
                        if inner == len(i)-1:
                            break
                        output_string += ", "
                    output_string += ')'
                    if k == len(input)-1:
                        break
                    output_string += ','
                output_string += ')'
        return output_string

In [24]:
def values_new(input, label):
        output_string = ''
        if (type(input) == int):
            output_string += str(input)
        elif type(input) == list:
            if type(input[0]) == int:
                output_string += f'{label}(0) <= ('
                for k, i in enumerate(input):
                    print(i)
                    output_string += str(i)
                    if k == len(input)-1:
                        break
                    output_string += ", "
                output_string += ');'

            elif type(input[0]) == list:
                for k, i in enumerate(input):
                    print(k)
                    output_string += f'{label}({k}) <= ('
                    for inner, j in enumerate(i):
                        output_string += str(j)
                        if inner == len(i)-1:
                            break
                        output_string += ", "
                    output_string += ');\n'
        return output_string

In [28]:
print(values([[2,3,4], [9,8,9]], 'input_weights'))

4


In [ ]:
a = 0.95 * 1 + 0.86
print(a)
b = 95 * 1 + 86
print(f"b = {b/10}")

1.81
b = 18.1


In [1]:
tanh = [-761,-757,-753,-748,-744,-739,-735,-730,-725,-721,-716,-711,-706,-701,-696,-691,-685,-680,-675,-669,-664,-658,-652,-646,-641,-635,-629,-623,-616,-610,-604,-597,-591,-584,-578,-571,-564,-558,-551,-544,-537,-529,-522,-515,-507,-500,-492,-485,-477,-469,-462,-454,-446,-438,-430,-421,-413,-405,-396,-388,-379,-371,-362,-353,-345,-336,-327,-318,-309,-300,-291,-282,-272,-263,-254,-244,-235,-226,-216,-206,-197,-187,-178,-168,-158,-148,-139,-129,-119,-109,-99,-89,-79,-69,-59,-49,-39,-29,-19,-9,0,9,19,29,39,49,59,69,79,89,99,109,119,129,139,148,158,168,178,187,197,206,216,226,235,244,254,263,272,282,291,300,309,318,327,336,345,353,362,371,379,388,396,405,413,421,430,438,446,454,462,469,477,485,492,500,507,515,522,529,537,544,551,558,564,571,578,584,591,597,604,610,616,623,629,635,641,646,652,658,664,669,675,680,685,691,696,701,706,711,716,721,725,730,735,739,744,748,753,757,761,765,769,773,777,781,785,789,793,796,800,804,807,811,814,817,821,824,827,830,833,836,839,842,845,848,851,853,856,859,861,864,866,869,871,874,876,878,880,883,885,887,889,891,893,895,897,899,901,903,905,906,908,910,912,913,915,917,918,920,921,923,924,926,927,928,930,931,932,934,935,936,937,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,957,958,959,960,961,961,962,963,964,964,965,966,966,967,968,968,969,969,970,971,971,972,972,973,973,974,974,975,975,976,976,977,977,978,978,978,979,979,980,980,980,981,981,981,982,982,983,983,983,983,984,984,984,985,985,985,986,986,986,986,987,987,987,987,988,988,988,988,989,989,989,989,989,990,990,990,990,990,991,991,991,991,991,991,992,992,992,992,992,992,992,993,993,993,993,993,993,993,993,994,994,994,994,994,994,994,994,994,995,995,995,995,995,995,995,995,995,995,995,996,996,996,996,996,996,996,996,996,996,996,996,996,996,996,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999]
tanh[100]
len(tanh)#

1101

In [ ]:
sig = [268,270,272,274,276,278,280,282,284,286,289,291,293,295,297,299,301,303,305,307,310,312,314,316,318,320,323,325,327,329,331,334,336,338,340,342,345,347,349,352,354,356,358,361,363,365,368,370,372,375,377,379,382,384,386,389,391,394,396,398,401,403,406,408,410,413,415,418,420,423,425,428,430,432,435,437,440,442,445,447,450,452,455,457,460,462,465,467,470,472,475,477,480,482,485,487,490,492,495,497,500,502,504,507,509,512,514,517,519,522,524,527,529,532,534,537,539,542,544,547,549,552,554,557,559,562,564,567,569,571,574,576,579,581,584,586,589,591,593,596,598,601,603,605,608,610,613,615,617,620,622,624,627,629,631,634,636,638,641,643,645,647,650,652,654,657,659,661,663,665,668,670,672,674,676,679,681,683,685,687,689,692,694,696,698,700,702,704,706,708,710,713,715,717,719,721,723,725,727,729,731,733,734,736,738,740,742,744,746,748,750,752,753,755,757,759,761,763,764,766,768,770,772,773,775,777,779,780,782,784,785,787,789,790,792,794,795,797,798,800,802,803,805,806,808,809,811,813,814,816,817,819,820,822,823,824,826,827,829,830,832,833,834,836,837,838,840,841,842,844,845,846,848,849,850,851,853,854,855,856,858,859,860,861,862,864,865,866,867,868,869,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,902,903,904,905,906,907,908,908,909,910,911,912,912,913,914,915,916,916,917,918,919,919,920,921,922,922,923,924,924,925,926,926,927,928,928,929,930,930,931,932,932,933,934,934,935,935,936,937,937,938,938,939,939,940,941,941,942,942,943,943,944,944,945,945,946,946,947,947,948,948,949,949,950,950,951,951,952,952,953,953,953,954,954,955,955,956,956,956,957,957,958,958,958,959,959,960,960,960,961,961,961,962,962,963,963,963,964,964,964,965,965,965,966,966,966,967,967,967,968,968,968,968,969,969,969,970,970,970,970,971,971,971,972,972,972,972,973,973,973,973,974,974,974,974,975,975,975,975,976,976,976,976,977,977,977,977,977,978,978,978,978,978,979,979,979,979,979,980,980,980,980,980,981,981,981,981,981,982,982,982,982,982,982,983,983,983,983,983,983,984,984,984,984,984,984,984,985,985,985,985,985,985,985,986,986,986,986,986,986,986,987,987,987,987,987,987,987,987,987,988,988,988,988,988,988,988,988,989,989,989,989,989,989,989,989,989,989,990,990,990,990,990,990,990,990,990,990,990,991,991,991,991,991,991,991,991,991,991,991,991,992,992,992,992,992,992,992,992,992,992,992,992,992,993,993,993,993,993,993,993,993,993,993,993,993,993,993,993,994,994,994,994,994,994,994,994,994,994,994,994,994,994,994,994,994,994,994,995,995,995,995,995,995,995,995,995,995,995,995,995,995,995,995,995,995,995,995,995,995,996,996,996,996,996,996,996,996,996,996,996,996,996,996,996,996,996,996,996,996,996,996,996,996,996,996,996,996,996,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,997,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,998,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999,999]
sig[151]

624

In [1]:
import numpy as np

In [2]:
a = np.array([0, 1, 2, 3])
b = np.array([[0, 4, 8, 2], [1, 5, 9, 3], [2, 6, 0, 4], [3, 7, 1, 5]])
np.dot(a, b)

array([14, 38, 12, 26])

In [33]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def lstm_unit(input_value, long_memory, short_memory):
        wlr1 = np.array([[500, -100], [300, 200]])
        wlr2 = np.array([[100, -300], [-200, 100]])
        blr1 = np.array([100, -200])

        # candidate long term memory/cell state weights and biases
        wpr1 = np.array([[400, -300], [300, 100]])
        wpr2 = np.array([[200, 500], [100, -200]])
        bpr1 = np.array([100, -100])

        # potential memory weights and biases
        wp1 = np.array([[600, -200], [-100, 300]])
        wp2 = np.array([[200, -400], [300, 100]])
        bp1 = np.array([-100, 100])

        # output weights and biases
        wo1 = np.array([[500, 100], [-300, 200]])
        wo2 = np.array([[400, -200], [200, 300]])
        bo1 = np.array([0, 100])

        # determine how much of long term memory to remember - forget gate
        long_remember_percent = sigmoid(np.dot(short_memory, wlr1)
                                              + np.dot(input_value,  wlr2) +
                                              blr1)
        # print(f"forget = {np.dot(short_memory, wlr1) + np.dot(input_value,  wlr2) +blr1}")
        print(np.dot(input_value,  wlr2))
        # calculate how much of potential memory should be remembered - Candidate gate
        potential_remember_percent = sigmoid(np.dot(short_memory,  wpr1)
                                              + np.dot(input_value,  wpr2) +
                                              bpr1)
        print(f"candidate = {np.dot(short_memory, wpr1)  + np.dot(input_value, wpr2) + bpr1}")
        # calculate candidate memory - Input gate
        potential_memory = np.tanh(np.dot(short_memory, wp1)
                                              + np.dot(input_value, wp2) +
                                              bp1)
        print(f"input = {np.dot(short_memory, wp1)+ np.dot(input_value, wp2) +bp1}")
        # update long term memory
        updated_long_memory = ((long_memory * long_remember_percent) +
                               (potential_remember_percent * potential_memory))
        print(f"new_long = {updated_long_memory}")
        # calculate new short term memory and how much of it to output - Output gate
        output_percent = sigmoid(np.dot(short_memory, wo1) + 
                                       np.dot(input_value, wo2) +
                                       bo1)
        print(f"output = {np.dot(short_memory, wo1) + np.dot(input_value, wo2) +bo1}")
        updated_short_memory = (np.tanh(updated_long_memory) * output_percent)
        print(f"new short = {updated_short_memory}")
        print(f"\n")

        return ([updated_long_memory, updated_short_memory])
    
def forward(input):
    long_memory = np.array([0, 0])
    short_memory = np.array([0, 0])
    # test inputs
    day1 = input[0]
    day2 = input[1]
    day3 = input[2]

    long_memory, short_memory = lstm_unit(day1, long_memory, short_memory)
    long_memory, short_memory = lstm_unit(day2, long_memory, short_memory)
    long_memory, short_memory = lstm_unit(day3, long_memory, short_memory)

    return short_memory

In [34]:
new_short = forward([[1.0, 0.5], [0.5, 1.0], [1.0, 1.0]])

forget = [ 100. -450.]
candidate = [350. 300.]
input = [ 250. -250.]
new_long = [ 1. -1.]
output = [500.  50.]
new short = [ 0.76159416 -0.76159416]


forget = [ 102.31883119 -478.47824679]
candidate = [ 376.1594156  -354.63766238]
input = [ 833.11590917 -380.79707798]
new_long = [ 2.00000000e+000 -9.61214143e-155]
output = [1009.27532476  223.8405844 ]
new short = [ 9.64027580e-001 -9.61214143e-155]


forget = [ 482.01379004 -496.40275801]
candidate = [785.61103203 -89.20827402]
input = [ 978.41654805 -392.80551602]
new_long = [ 3.0000000e+00 -1.8085847e-39]
output = [1082.01379004  296.40275801]
new short = [ 9.95054754e-01 -1.80858470e-39]




In [26]:
print(np.dot(np.array([1, 3]), np.array([[1, 2], [2,3]])) + np.dot(np.array([1, 3]), np.array([[1, 2], [2,3]])) + np.array([1, 5]))

[15 27]
